In [ ]:
import boto3
qclient = boto3.client('qldb')

In [ ]:
resp = qclient.create_ledger(
    Name='vehicle-registration',
    PermissionsMode='STANDARD',
    DeletionProtection=False
)

print(resp)

In [ ]:
qclient.list_ledgers()

In [ ]:
# Once the ledger is created you can work with it via the driver. Install
# it via pip install pyqldb

In [ ]:
from pyqldb.driver.qldb_driver import QldbDriver
qldb_driver = QldbDriver('vehicle-registration')

In [ ]:
for table in qldb_driver.list_tables():
    print(table)

In [ ]:
def create_table_fn(table_name):
    def create_table(transaction_executor):
        print("Creating a table")
        transaction_executor.execute_statement("Create TABLE " + table_name)
        
    return create_table
    


In [ ]:
#create_VehicleRegistration = create_table_fn('VehicleRegistration')
#create_Vehicle = create_table_fn('Vehicle')
#create_Person = create_table_fn('Person')
#create_DriversLicense = create_table_fn('DriversLicense')
qldb_driver.execute_lambda(lambda executor: create_table_fn('VehicleRegistration')(executor))
qldb_driver.execute_lambda(lambda executor: create_table_fn('Vehicle')(executor))
qldb_driver.execute_lambda(lambda executor: create_table_fn('Person')(executor))
qldb_driver.execute_lambda(lambda executor: create_table_fn('DriversLicense')(executor))

In [ ]:
def create_index_fn(stmt):
    def create_index(transaction_executor):
        transaction_executor.execute_statement(stmt)
    return create_index

In [ ]:
qldb_driver.execute_lambda(lambda executor: create_index_fn("CREATE INDEX ON VehicleRegistration (VIN)")(executor))
qldb_driver.execute_lambda(lambda executor: create_index_fn("CREATE INDEX ON VehicleRegistration (LicensePlateNumber)")(executor))
qldb_driver.execute_lambda(lambda executor: create_index_fn("CREATE INDEX ON Vehicle (VIN)")(executor))
qldb_driver.execute_lambda(lambda executor: create_index_fn("CREATE INDEX ON Person (GovId)")(executor))
qldb_driver.execute_lambda(lambda executor: create_index_fn("CREATE INDEX ON DriversLicense (LicenseNumber)")(executor))
qldb_driver.execute_lambda(lambda executor: create_index_fn("CREATE INDEX ON DriversLicense (PersonId)")(executor))


In [ ]:
resp = qclient.delete_ledger(
    Name='vehicle-registration'
)

print(resp)